### Implement the scaled dot product attention with Wq,Wk,Wv matrices 

Compute key, value, queue values --> compute attention score for each of the words to measure similarity/importance --> scale the attention score 

In [9]:
import torch 
import numpy as np 

In [10]:
inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

In [10]:
x= inputs[1] 
d_in = inputs. shape[1]     # input dimension 
d_out= 2    # hidden dimension

In [11]:
torch.manual_seed(123)

W_query = torch.nn.Parameter(torch.rand(d_in, d_out))
W_key   = torch.nn.Parameter(torch.rand(d_in, d_out))
W_value = torch.nn.Parameter(torch.rand(d_in, d_out))   # weight matrices are vocab size x hidden dimension

In [12]:
query_2= x @ W_query 
key_2= x @ W_key 
value_2= x @W_value 

# output: 1 x 2 to get the value for query, key and value 
print(query_2)

tensor([0.4306, 1.4551], grad_fn=<SqueezeBackward4>)


In [ ]:
# for each input, we need the attention from other inputs of Wk, and Wv 
keys= inputs @ W_key 
values= inputs @ W_value  # dim: 6 x 2 
print (keys)
print (keys.shape)

# each row is the key/value of each word (including our current target word)

tensor([[0.3669, 0.7646],
        [0.4433, 1.1419],
        [0.4361, 1.1156],
        [0.2408, 0.6706],
        [0.1827, 0.3292],
        [0.3275, 0.9642]], grad_fn=<MmBackward0>)
torch.Size([6, 2])


**Computing the attention score**  
-  Each word has an attention score for every input word. For example, $\omega_{22}$ is the second word's key value given the target word is the 2nd word 
-  Compute the dot product between key (of non-input) and the query of input so we get the similarity between these 2 words 

In [ ]:
# sample for W22 

keys_2= keys[1] # retrive the key value for the second word 
attn_score_22 = query_2.dot(keys_2)
print(attn_score_22)

tensor(1.8524, grad_fn=<DotBackward0>)


We can compute this attention score between the input word and every other word in the input sentence 

In [15]:
attn_scores= query_2 @ keys.T 
print(attn_scores)

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440],
       grad_fn=<SqueezeBackward4>)


Scaling the attention by dividing each term by d_k ** 0.5, so we avoid too large dot products that lead to very small gradients 

In [ ]:
d_k= keys.shape[-1] # column size= 2 
attn_weights= torch.softmax(attn_scores/ d_k **0.5, dim=-1)
print (attn_weights)    # 1x 6 

# the attention score for each word related to the input word because we multiplied this dot product with input word's query vector 

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820],
       grad_fn=<SoftmaxBackward0>)


Compute the context vector to get the relative importance   
-  multiply the relative attention score with corresponding value block 

In [ ]:
context_vector= attn_weights @ values # dim: 1x 2 
print(context_vector)

tensor([0.3590, 0.9117], grad_fn=<SqueezeBackward4>)


We can extend this idea to a scaled dot product attention class where the operations are performed on each row. So instead of 1 x2 context vector for a single row(single input word), we'll have 6x2. Operations are done on each row.

#### Experiemntation 

In [16]:
import torch.nn as nn 
d_in= 3 
d_out =2 

class SelfAttention_v2(nn.Module):

    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

    def forward(self, x):
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
        
        attn_scores = queries @ keys.T
        print(f"Attention Score:\n{attn_scores}\n")
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        print(f"Attention Weight Matrix:\n{attn_weights}")

        context_vec = attn_weights @ values
        return context_vec

torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in, d_out)
print(f"Context Vector:\n{sa_v2(inputs)}")

Attention Score:
tensor([[ 0.2899,  0.0716,  0.0760, -0.0138,  0.1344, -0.0511],
        [ 0.4656,  0.1723,  0.1751,  0.0259,  0.1771,  0.0085],
        [ 0.4594,  0.1703,  0.1731,  0.0259,  0.1745,  0.0090],
        [ 0.2642,  0.1024,  0.1036,  0.0186,  0.0973,  0.0122],
        [ 0.2183,  0.0874,  0.0882,  0.0177,  0.0786,  0.0144],
        [ 0.3408,  0.1270,  0.1290,  0.0198,  0.1290,  0.0078]],
       grad_fn=<MmBackward0>)

Attention Weight Matrix:
tensor([[0.1921, 0.1646, 0.1652, 0.1550, 0.1721, 0.1510],
        [0.2041, 0.1659, 0.1662, 0.1496, 0.1665, 0.1477],
        [0.2036, 0.1659, 0.1662, 0.1498, 0.1664, 0.1480],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.1661, 0.1564],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.1585],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)
Context Vector:
tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
  